<a href="https://www.kaggle.com/code/tarktunataalt/sarima-holt-winters-for-monthly-car-sales?scriptVersionId=184411438" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
df=read.csv("/kaggle/input/newcarsalesnorway/norway_new_car_sales_by_month.csv")


In [ ]:
str(df)

In [ ]:
df=subset(df,select=c(Quantity))


In [ ]:
str(df)

In [ ]:
ts_df=ts(df,frequency=12,start=c(2007,01,01),end=c(2017,01,01))
boxplot(ts_df, horizontal = TRUE, xlab = "", ylab = "Values",
        main = "Monthly Car Sales BoxPlot")

The boxplot shows that there are some lower outliers in the dataset, and the majority of the data is concentrated between approximately 10,000 and 13,000. The median value is around 11,385.

In [ ]:
ts.plot(ts_df, main = "Monthly Car Sales Time Series Plot")


The time series plot shows a noticeable decline around 2009, followed by a general upward trend in the data from 2010 onwards, with significant fluctuations and seasonality observed throughout the entire period.

In [ ]:
plot(decompose(ts_df))

The decomposition plot of the additive time series reveals the following:


Observed: The original time series data with noticeable fluctuations.

Trend: A clear declining trend until around 2010, followed by a steady increase.

Seasonal: Regular seasonal patterns that repeat annually.

Random: The residual component capturing irregular fluctuations not explained by the trend or seasonal components.


# EXPONENTIAL SMOOTHING

Based on the decomposition graph, I chose additive seasonality for the Holt-Winters model. The consistent seasonal pattern with constant magnitude fluctuations supports the additive component, as it indicates that seasonal variations do not change proportionally with the series level.


In [ ]:
library(forecast)
holtwintermodel=HoltWinters(ts_df,seasonal="additive")
plot(holtwintermodel)


In the graph where the Holt-Winters filtering is applied, the black line represents the observed data, while the red line shows the values predicted by the model. The model captures the general trends and seasonality of the data quite well. However, at certain periods, there are deviations between the predictions and the observed values, indicating that the model does not always capture all fluctuations.

In [ ]:
holtwintermodel

# SARIMA

In [ ]:
library(tseries)

## RAW DATA

In [ ]:
library(ggplot2)
ggAcf(ts_df, lag.max = 50) + ggtitle("ACF of Raw Data")
ggPacf(ts_df, lag.max = 50) + ggtitle("PACF of Raw Data")


The ACF plot shows a gradually decreasing structure over many lags, while the PACF plot exhibits a significant drop in the first few lags and then values close to zero for most lags. This indicates that the series is non-stationary and likely contains a trend or seasonality. These patterns observed in the ACF and PACF plots suggest that the data needs to be made stationary before conducting further analysis or modeling.

## DATA DIFFERENCED ONCE NON-SEASONALLY

In [ ]:
library(forecast)
library(lmtest)
library(zoo)
automodel=auto.arima(ts_df,stepwise=FALSE)
coeftest(automodel)


In [ ]:
automodel

Auto-ARIMA suggests an ARIMA(2,1,0)(1,0,1) model.

In [ ]:
checkresiduals(automodel)

The residuals from the ARIMA(2,1,0)(1,0,1)[12] model appear to be roughly normally distributed, with no significant autocorrelation, indicating a good fit. The histogram and ACF plot confirm that the residuals behave like white noise, suggesting the model effectively captures the underlying data structure.

In [ ]:
tsdiag(automodel)

In [ ]:
Box.test(automodel$residuals,lag=6)
Box.test(automodel$residuals,lag=12)
Box.test(automodel$residuals,lag=24)
Box.test(automodel$residuals,lag=36)

The residual diagnostics plot indicates that the standardized residuals fluctuate around zero without any apparent pattern, and the ACF of residuals shows no significant autocorrelation. The p-values from the Ljung-Box tests are all well above 0.05, suggesting that there is no significant autocorrelation in the residuals at various lags. This implies that the ARIMA model has adequately captured the underlying patterns in the data, and the residuals behave like white noise.

In [ ]:
shapiro.test(automodel$residuals)

The Shapiro-Wilk normality test for the residuals of automodel (ARIMA(2,1,0)(2,1,0)[12]) yields a W value of 0.99186 and a p-value of 0.7026.

H0 (null hypothesis): The residuals are normally distributed.
H1 (alternative hypothesis): The residuals are not normally distributed.
Since the p-value (0.7026) is greater than 0.05, we fail to reject the null hypothesis (H0). This suggests that the residuals of automodel (ARIMA(2,1,0)(2,1,0)[12]) are normally distributed.

In [ ]:
diff1=diff(ts_df,1)
ggAcf(diff1,lag.max=50) + ggtitle("ACF of Data Differenced Once Non-Seasonally")
ggPacf(diff1,lag.max=50) + ggtitle("PACF of Data Differenced Once Non-Seasonally")

The ACF and PACF plots for the first differenced data show that the non-seasonal part of the series is stationary, as indicated by the quick drop in autocorrelations. However, there are still significant spikes at seasonal lags, suggesting that the seasonal part of the series is not yet stationary. This indicates the need for seasonal differencing to achieve stationarity in the seasonal component.

## DATA DIFFERENCED ONCE NON-SEASONALLY AND ONCE SEASONALLY

In [ ]:
diff112=diff(diff1,12)
ggAcf(diff112,lag.max=50) + ggtitle("ACF of Data Differenced Once Non-Seasonally and Once Seasonally")
ggPacf(diff112,lag.max=50) + ggtitle("PACF of Data Differenced Once Non-Seasonally and Once Seasonally")

The selected models are worth testing because they better capture the autocorrelation structures observed in the ACF and PACF plots and explain the residuals more effectively. The ARIMA(2,1,0) model is chosen because the ACF and PACF plots show a rapid decline after the first few lags, indicating that two autoregressive (AR) terms might be sufficient. The ARIMA(0,1,1) model is selected due to the presence of positive autocorrelation at some lags in the ACF plot, suggesting that a moving average (MA) term could be significant. The ARIMA(2,1,1) model is considered because the combination of AR and MA terms might better capture the autocorrelation in the data and explain the residuals more comprehensively. For the seasonal model, ARIMA(2,1,0)[12] is chosen as it can sufficiently explain the seasonal autocorrelation observed every 12 lags and capture the annual cycles. These models collectively help in better modeling the different autocorrelation structures observed in the data.  

## COMPARISON OF ARIMA MODELS

In [ ]:
library(lmtest)
model1=arima(ts_df,order=c(2,1,0),seasonal=list(order=c(2,1,0),period=12))
model1
coeftest(model1)


All the coefficients in the ARIMA(2,1,0)(2,1,0)[12] model are statistically significant, as indicated by their very low p-values (all less than 0.001). This suggests that the model's parameters are indeed contributing meaningfully to the model. Therefore, the validity of the model cannot be rejected based on the significance of the coefficients.

In [ ]:
model2=arima(ts_df,order=c(0,1,1),seasonal=list(order=c(2,1,0),period=12))
model2
coeftest(model2)


All the coefficients in the ARIMA(0,1,1)(2,1,0)[12] model are statistically significant, as indicated by their very low p-values (all less than 0.001). This suggests that the model's parameters are indeed contributing meaningfully to the model. Therefore, the validity of the model cannot be rejected based on the significance of the coefficients.

In [ ]:
model3=arima(ts_df,order=c(2,1,1),seasonal=list(order=c(2,1,0),period=12))
model3
coeftest(model3)


In the ARIMA(2,1,1)(2,1,0)[12] model, the MA1 coefficient is not statistically significant (p-value = 0.7617), indicating that this parameter does not contribute meaningfully to the model. Therefore, the model's validity is questionable due to the insignificance of the MA1 coefficient, and it will be excluded from further evaluation.

In [ ]:
tsdiag(model1)

For Model 1 (ARIMA(2,1,0)(2,1,0)[12]), the diagnostic plots of the standardized residuals show no obvious patterns, indicating that the residuals are randomly distributed around zero. The ACF plot of the residuals shows that all autocorrelations are within the confidence bounds, suggesting no significant autocorrelation in the residuals. The p-values from the Ljung-Box test are all above 0.05, indicating that there is no significant autocorrelation at any lag, and the residuals behave like white noise. These diagnostics suggest that Model 1 is well-specified and fits the data adequately.

In [ ]:
tsdiag(model2)

For Model 2 (ARIMA(0,1,1)(2,1,0)[12]), the diagnostic plots of the standardized residuals show some patterns, indicating that the residuals might not be randomly distributed around zero. The ACF plot of the residuals shows that some autocorrelations are outside the confidence bounds, suggesting significant autocorrelation in the residuals. The p-values from the Ljung-Box test are consistently below 0.05, indicating significant autocorrelation at various lags. This suggests that the model has not adequately captured the underlying patterns in the data, and its validity is questionable. Therefore, Model 2 is not a suitable model based on the residual diagnostics and will be excluded from further evaluation.

In [ ]:
Box.test(model1$residuals,lag=6)
Box.test(model1$residuals,lag=12)
Box.test(model1$residuals,lag=24)
Box.test(model1$residuals,lag=36)

For Model 1 (ARIMA(2,1,0)(2,1,0)[12]), the p-values from the Box-Pierce tests are all well above 0.05 (e.g., p-value = 0.7478 for 6 lags, p-value = 0.8703 for 12 lags, p-value = 0.8276 for 24 lags, and p-value = 0.7608 for 36 lags). These high p-values indicate no significant autocorrelation in the residuals, further supporting the model's validity and adequacy.

In [ ]:
Box.test(model2$residuals,lag=6)
Box.test(model2$residuals,lag=12)
Box.test(model2$residuals,lag=24)
Box.test(model2$residuals,lag=36)

For Model 2 (ARIMA(0,1,1)(2,1,0)[12]), the residual diagnostics and Box-Pierce test results support its invalidity. The residuals show significant autocorrelation, with all p-values below 0.05. This confirms that the model has not adequately captured the data patterns and is unsuitable, warranting exclusion from further evaluation

In [ ]:
checkresiduals(model1)

In [ ]:
shapiro.test(model1$residuals)

The Shapiro-Wilk normality test for the residuals of model1 (ARIMA(2,1,0)(2,1,0)[12]) yields a W value of 0.9899 and a p-value of 0.5193.

H0 (null hypothesis): The residuals are normally distributed.
H1 (alternative hypothesis): The residuals are not normally distributed.
Since the p-value (0.5193) is greater than 0.05, we fail to reject the null hypothesis (H0). This suggests that the residuals of model1 (ARIMA(2,1,0)(2,1,0)[12]) are normally distributed.

## SELECTION OF THE FINAL ARIMA MODEL

| Model               | All Parameters Significant? | Significant Autocorrelation in Residuals? | Residuals Normally Distributed? | P Values Above Threshold? | Log Likelihood Values | AIC Values              |
|---------------------|-----------------------------|--------------------------------------------|----------------------------------|----------------------------|-----------------------|-------------------------|
| ARIMA(2,1,0)(1,0,1) | YES ✅                       | NO ✅                                      | YES ✅                           | YES ✅                      | -1005.82              | 2021.65                 |
| ARIMA(2,1,0)(2,1,0) | YES ✅                       | NO ✅                                      | YES ✅                           | YES ✅                      | -912.74               | 1835.47                 |
| ARIMA(0,1,1)(2,1,0) | YES ✅                       | YES ❌                                     | YES ✅                           | NO ❌                       | EXCLUDED ⛔           | EXCLUDED ⛔             |
| ARIMA(2,1,1)(2,1,0) | NO ❌                        | EXCLUDED ⛔                                | EXCLUDED ⛔                      | EXCLUDED ⛔                 | EXCLUDED ⛔           | EXCLUDED ⛔             |


Based on the analysis, ARIMA(2,1,0)(2,1,0) was selected due to its lower AIC value of 1835.47, indicating a better fit compared to the other models. Although ARIMA(2,1,0)(1,0,1) is also a valid model with significant parameters, no significant autocorrelation in the residuals, and normally distributed residuals with p-values above the threshold, the lower AIC value of ARIMA(2,1,0)(2,1,0) supports its selection. Additionally, ARIMA(2,1,0)(2,1,0) has all significant parameters, no significant autocorrelation in the residuals, and the residuals are normally distributed with p-values above the threshold, further supporting its validity and adequacy for the data.

In [ ]:
forecasts=forecast(model1,h=12)
plot(forecasts)

In [ ]:
forecasts

The forecasts from the ARIMA(2,1,0)(2,1,0)[12] model indicate that the model fits the data well, capturing both the seasonal fluctuations and the overall trend effectively. For instance, the point forecast for February 2017 is 12727.25, with an 80% confidence interval ranging from 11323.87 to 14130.63 and a 95% confidence interval ranging from 10580.96 to 14873.53. Although the prediction intervals widen as we move further into the forecast period, the model successfully reflects the seasonal components and the overall trend. This demonstrates that the model effectively captures both seasonality and long-term trends, making it reliable for forecasting future data.